In [4]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_curve, auc, log_loss

In [5]:
data = pd.read_csv('../data/catboost/data/train_data1.csv')
train_data = data[data['eval_set'] != 'test']
valid_data = data[data['eval_set'] == 'test']

In [8]:
train_features = np.array(train_data.iloc[:,3:])
train_label = train_data['reordered'].values
cat_features = [0,1,2]

In [9]:
valid_features = np.array(valid_data.iloc[:,3:])
valid_label = valid_data['reordered'].values
cat_features = [0,1,2]

In [10]:
params = {}
params['iterations'] = 50
params['learning_rate'] = 0.03
params['depth'] = 6
params['random_seed'] = 108
params['thread_count'] = 7
params['auto_stop_pval'] = 10

In [11]:
model = CatBoostClassifier(**params)

In [12]:
model.fit(train_features, train_label, cat_features, verbose=True)

Borders generated
0:	learn 0.6689203758passed: 6.07 sec	total: 5m 45s	remaining: 4h 42m
1:	learn 0.6461972467passed: 6.19 sec	total: 5m 51s	remaining: 2h 20m 36s
2:	learn 0.625033109passed: 5.19 sec	total: 5m 56s	remaining: 1h 33m 8s
3:	learn 0.6058615472passed: 5.43 sec	total: 6m 2s	remaining: 1h 9m 24s
4:	learn 0.5879741224passed: 5.31 sec	total: 6m 7s	remaining: 55m 7s
5:	learn 0.5711978244passed: 5.65 sec	total: 6m 13s	remaining: 45m 36s
6:	learn 0.5560914795passed: 6.23 sec	total: 6m 19s	remaining: 38m 50s
7:	learn 0.5417919034passed: 6.04 sec	total: 6m 25s	remaining: 33m 43s
8:	learn 0.5286838781passed: 5.2 sec	total: 6m 30s	remaining: 29m 39s
9:	learn 0.5165469067passed: 5.6 sec	total: 6m 36s	remaining: 26m 24s
10:	learn 0.505243332passed: 5.04 sec	total: 6m 41s	remaining: 23m 42s
11:	learn 0.4948499978passed: 6.74 sec	total: 6m 47s	remaining: 21m 31s
12:	learn 0.4852307201passed: 5.64 sec	total: 6m 53s	remaining: 19m 37s
13:	learn 0.4762752111passed: 5.89 sec	total: 6m 59s	rema

In [13]:
predictions = model.predict_proba(valid_features)

In [14]:
fpr, tpr, thresh = roc_curve(valid_label, predictions[:,1])
2*auc(fpr,tpr) - 1

0.49834976847589707

In [15]:
data = pd.read_csv('../data/catboost/data/score_data1.csv')
features = np.array(data.iloc[:,3:])
label = data['reordered'].values

In [16]:
predictions = model.predict_proba(features)

In [17]:
fpr, tpr, thresh = roc_curve(label, predictions[:,1])
2*auc(fpr,tpr) - 1

0.47921359936064722

In [19]:
driver = data[['user_id','product_id']]
driver['catboost1'] = predictions[:,1]

/home/ubuntu/.software/anaonda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
driver.to_csv('../data/catboost/catboost1.csv', index=False)